In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
from plotly.subplots import make_subplots
import pandas as pd
import os
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot, plot
import math
import copy
import numpy as np
from IPython.display import display, clear_output
from ipywidgets import widgets, Button, HBox, VBox
from plotly.colors import DEFAULT_PLOTLY_COLORS

import random



In [ ]:
def frequency_to_note(frequency):
    # define constants that control the algorithm
    NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'] # these are the 12 notes in each octave
    OCTAVE_MULTIPLIER = 2 # going up an octave multiplies by 2
    KNOWN_NOTE_NAME, KNOWN_NOTE_OCTAVE, KNOWN_NOTE_FREQUENCY = ('A', 4, 440) # A4 = 440 Hz

    # calculate the distance to the known note
    # since notes are spread evenly, going up a note will multiply by a constant
    # so we can use log to know how many times a frequency was multiplied to get from the known note to our note
    # this will give a positive integer value for notes higher than the known note, and a negative value for notes lower than it (and zero for the same note)
    note_multiplier = OCTAVE_MULTIPLIER**(1/len(NOTES))
    frequency_relative_to_known_note = frequency / KNOWN_NOTE_FREQUENCY
    distance_from_known_note = math.log(frequency_relative_to_known_note, note_multiplier)

    # round to make up for floating point inaccuracies
    distance_from_known_note = round(distance_from_known_note)

    # using the distance in notes and the octave and name of the known note,
    # we can calculate the octave and name of our note
    # NOTE: the "absolute index" doesn't have any actual meaning, since it doesn't care what its zero point is. it is just useful for calculation
    known_note_index_in_octave = NOTES.index(KNOWN_NOTE_NAME)
    known_note_absolute_index = KNOWN_NOTE_OCTAVE * len(NOTES) + known_note_index_in_octave
    note_absolute_index = known_note_absolute_index + distance_from_known_note
    note_octave, note_index_in_octave = note_absolute_index // len(NOTES), note_absolute_index % len(NOTES)
    note_name = NOTES[note_index_in_octave]
    return note_name

In [ ]:
collections = {"sm":"Scherbaum Mshavanadze",
               "guria":"Teach Yourself Gurian Songs",
               "megrelia":"Teach Yourself Megrelian Songs"}

collection_directories = {"sm":
                          ["GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919",
                           "GVM017_ChvenMshvidobaTake2_Ozurgeti_ShalvaChemo2016_20160713",
                           "GVM019_DaleKojas_DidgoriVillage_Didgori_20160707",
                           "GVM031_EliaLrde_LakhushdiVillage_MuradGigoGivi_20160819",
                           "GVM097_KristeAghsdga_LakhushdiVillage_MuradGigoGivi_20160819"],
                          "guria":
                          ["Adila-Alipasha",
                           "Indi-Mindi",
                           'Mival Guriashi (1)' ,
                           'Pikris Simghera',
                           "Alaverdi",
                           "K'alos Khelkhvavi",
                           'Mival Guriashi (2)' , 
                           "Sabodisho",
                           "Khasanbegura",     
                           "Mok'le Mravalzhamieri",
                           'Sadats Vshobilvar',
                           "Beri Ak'vans Epareba", 
                           "Lat'aris Simghera",    
                           "Mts'vanesa Da Ukudosa", 
                           "Shermanduli",
                           "Brevalo",             
                           "Manana",         
                           'Nanina (1)',      
                           "Shvidk'atsa",
                           "Chven-Mshvidoba",    
                           "Maq'ruli",               
                           'Nanina (2)',          
                           'Supris Khelkhvavi',
                           'Didi Khnidan',     
                           "Masp'indzelsa Mkhiarulsa", 
                           "Orira",                
                           "Ts'amok'ruli",
                           "Gakhsovs, T'urpa",
                           "Me-Rustveli",        
                           "P'at'ara Saq'varelo"]}

directories = collection_directories["sm"]

data_dir = "/Akamai/Voice/data/pitches-vuv-new/"
working_coll = collections["sm"]
larynx = True
working_song = "GVM017_ChvenMshvidobaTake2_Ozurgeti_ShalvaChemo2016_20160713"

algos = ['boersma', 'noll', 'crepe', 'maddox', 'hermes', 'yin'] # Note: later add praat
data3 = {}
locations = {}

for algo in algos:
    data3[algo] = {}
    locations[algo] = {}
    for direct in directories:
        data3[algo][direct] = {
            "mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }
        locations[algo][direct] = {
            "mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }
        


def separate(adir, algo):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

def load_songs():
    global data_dir, working_coll, working_song
    for algorithm in sorted(os.listdir(data_dir)):
        if algorithm in data3:
            print(algorithm)
            for collection in sorted(os.listdir(f"{data_dir}{algorithm}")):
                if collection != working_coll:
                    continue
                print(" ", collection)
                sm = (collection == "Scherbaum Mshavanadze")
                suffix = 'ALRX' if larynx and sm else 'AHDS' 
                for song in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}")):
                    if song != working_song:
                        continue
                    print("  ", song)
                    for location in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}/{song}")):
                        if (location[-4:] == '.txt'):
                            x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{location}", algo=algorithm)

                            freq = np.array(y).flatten()
                            i = 0
                            while i < len(freq):
                                sumFreq = freq[i]
                                a = 1
                                while i+a < len(freq) and freq[i+a] <= freq[i]*1.07 and freq[i+a] >= freq[i]*0.94:
                                    sumFreq += freq[i+a]
                                    a += 1

                                if a > 6 or i+a == len(freq)-1:
                                    for q in range(i,i+a+1):
                                        if q < len(freq):
                                            freq[q] = sumFreq/a
                                    i += a
                                else:
                                    freq[i] = 0
                                    i += 1  

                            notes = []
                            for i in range(0,len(freq)):
                                if freq[i] > 0:
                                    notes.append(frequency_to_note(freq[i]))
                                else:
                                    notes.append('N/A')

                            if not suffix in location:
                                data3[algorithm][song]['mix'] = (x, y, freq)
                                locations[algorithm][song]['mix'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                            else:
                                name_tag = location[(location.index(suffix) + 3):(location.index(suffix) + 6)]

                                if (name_tag[1] == '1' and not sm) or (name_tag[1] == '3' and sm):
                                    data3[algorithm][song]['bass'] = (x, y, freq)
                                    locations[algorithm][song]['bass'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"

                                elif name_tag[1] == '2':
                                    data3[algorithm][song]['middle'] = (x, y, freq)
                                    locations[algorithm][song]['middle'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                                else:
                                    data3[algorithm][song]['top'] = (x, y, freq)
                                    locations[algorithm][song]['top'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"

    print("\nLoaded song data from files into dictionary")
                       
load_songs()                                
                                

In [ ]:
selectedPoints = {}
targetSong = ""
traceIds = {}
targetAlgorithm = ""
undoState = []
selectionXrange = []
selectionYrange = []
   
def selection_fn(trace,points,selector):
    global targetSong, selectedPoints, selectionXrange, selectionYrange
    selectionXrange = selector.xrange
    selectionYrange = selector.yrange
    splitArray = points.trace_name.split(": ")
    algoName = splitArray[0]
    voice = splitArray[1]
    if(len(trace.selectedpoints) != 0):                
        selectedPoints[algoName+"-"+voice] = trace.selectedpoints
    ##print(points)
    #print(trace.selectedpoints)
    
def graph(song, targetAlgo, reload=False, type1 = 1):
    global targetSong, targetAlgorithm, undoState
    targetSong = song
    targetAlgorithm = targetAlgo
    print("target algo updated: " + targetAlgorithm)
    if reload:
        load_songs()
    print("plotting data from dictionary")
    traces = []
    type_list = []
    traceId = 0
    for algo, collection in data3.items():
        for name, part in collection.items():
            for audio_type, res in part.items():
                if name != song:
                    continue

                try:
                    trace = go.Scattergl(
                                x = res[0],
                                y = res[type1],
                                name=f"{algo}: {audio_type}",
                                mode="markers",
                                visible= (True if audio_type == "bass" else False)
                            )

                    traceIds[algo+"-"+audio_type] = traceId
                    traceId += 1
                    traces.append(trace)
                    type_list.append(audio_type)
                except:
                    print(f"{algo}: {audio_type} not available")

    layout = go.Layout(title='Activity Heatmap')

    figure = go.Figure(data=traces, layout=layout)

    fig = go.FigureWidget(figure)

    for i in range(0,len(traces)):
        fig.data[i].on_selection(selection_fn)

    buttons = []
    labels = ['bass', 'middle', 'top', 'mix']
    for i, label in enumerate(labels):
        visibility = [label==current_type for current_type in type_list]
        button = dict(
            label = label,
            method = "update",
            args = [{ 'visible': visibility}]
        )

        buttons.append(button)

    updatemenus = list([
        dict(active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x = 0.005,
            y = 1.06,
        )
    ])

    fig.update_layout(
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
        ),
        margin=dict(l=0, r=0, t=100, b=0)
    )


    fig.update_traces(
        marker=dict(size=3),
        selector=dict(mode='markers')
    )


    fig['layout']['title'] = "Fixing " + targetAlgorithm + " for " + song
    fig['layout']['width'] = 900
    fig['layout']['height'] = 500
    fig['layout']['showlegend'] = True
    fig['layout']['updatemenus'] = updatemenus
    
    undoState = copy.deepcopy(fig.data)


    def update_axes(Octave):
        print("Correct octave updated: " + Octave)
        
        
    def on_button_octave(x):
        global targetAlgorithm, targetSong
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName != targetAlgorithm:
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                if len(selectedPoints[algo]) > 0:
                    selectedpoints = list(selectedPoints[algo])
                    if(len(selectedPoints[algo]) != 0):    
                        i = selectedPoints[algo][0]
                        #print(data3[algoName][targetSong][voice][2])
                        while True:
                            selectedpoints.append(i)
                            if data3[algoName][targetSong][voice][2][i-1] != data3[algoName][targetSong][voice][2][i]:
                                break;
                            i = i-1
                        i = selectedPoints[algo][-1]
                        while True:
                            selectedpoints.append(i)
                            if data3[algoName][targetSong][voice][2][i+1] != data3[algoName][targetSong][voice][2][i]:
                                break;
                            i = i+1
                    selectedPoints[algo] = tuple(selectedpoints)
                    for point in selectedPoints[algo]:
                        if fig.data[traceIds[targetAlgorithm+"-"+voice]].y[point] > 0 and fig.data[traceIds[algo]].y[point] > 0:
                            distances = list()
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-newData[point]))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*4)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/4)))

                            minimum = distances.index(min(distances))
                            if minimum == 0:
                                newData[point] = newData[point]
                            if minimum == 1:
                                newData[point] = newData[point]*2
                            if minimum == 3:
                                newData[point] = newData[point]*3
                            if minimum == 5:
                                newData[point] = newData[point]*4
                            if minimum == 2:
                                newData[point] = newData[point]/2
                            if minimum == 4:
                                newData[point] = newData[point]/3
                            if minimum == 6:
                                newData[point] = newData[point]/4
                            
                fig.data[traceIds[targetAlgorithm+"-"+voice]].y = newData
                
    def on_button_octave_selected(x):
        global targetAlgorithm, targetSong
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName != targetAlgorithm:
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                if len(selectedPoints[algo]) > 0:
                    for point in selectedPoints[algo]:
                        if fig.data[traceIds[targetAlgorithm+"-"+voice]].y[point] > 0 and fig.data[traceIds[algo]].y[point] > 0:
                            distances = list()
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-newData[point]))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*4)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/4)))

                            minimum = distances.index(min(distances))
                            if minimum == 0:
                                newData[point] = newData[point]
                            if minimum == 1:
                                newData[point] = newData[point]*2
                            if minimum == 3:
                                newData[point] = newData[point]*3
                            if minimum == 5:
                                newData[point] = newData[point]*4
                            if minimum == 2:
                                newData[point] = newData[point]/2
                            if minimum == 4:
                                newData[point] = newData[point]/3
                            if minimum == 6:
                                newData[point] = newData[point]/4
                            
                fig.data[traceIds[targetAlgorithm+"-"+voice]].y = newData

    def on_button_delete(x):
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            newData = np.array(fig.data[traceIds[algo]].y)
            if len(selectedPoints[algo]) != 0:
                for point in selectedPoints[algo]:
                    newData[point] = 0
            fig.data[traceIds[algo]].y = newData
    
    def on_button_delete_note(x):
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            newData = np.array(fig.data[traceIds[algo]].y)
            if len(selectedPoints[algo]) != 0:
                selectedpoints = list(selectedPoints[algo])  
                i = selectedPoints[algo][0]
                print(data3[algoName][targetSong][voice][2])
                while True:
                    selectedpoints.append(i)
                    if data3[algoName][targetSong][voice][2][i-1] != data3[algoName][targetSong][voice][2][i]:
                        break;
                    i = i-1
                i = selectedPoints[algo][-1]
                while True:
                    selectedpoints.append(i)
                    if data3[algoName][targetSong][voice][2][i+1] != data3[algoName][targetSong][voice][2][i]:
                        break;
                    i = i+1
                selectedPoints[algo] = tuple(selectedpoints)
                for point in selectedPoints[algo]:
                    newData[point] = 0
                fig.data[traceIds[algo]].y = newData
            
    def on_button_postprocess(x):        
        global targetSong, targetAlgorithm
        undoState = copy.deepcopy(fig.data)
        print("Target Algorithm post-processing starting...")
        #fig.restyle(fig, 'selectedpoints', null)
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            fullName = algoName+"-"+voice
            if algoName == targetAlgorithm:                
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                for i in range(0, len(fig.data[traceIds[algoName+"-"+voice]].y)):
                    if data3[algoName][targetSong][voice][2][i] == 0 and data3[algoName][targetSong][voice][1][i] != 0:
                        newData[i] = -1                
                fig.data[traceIds[algoName+"-"+voice]].y = newData
        print("Target Algorithm post-processing done.")
    
    def on_button_save(x):
        global targetSong
        print("Beginning saving files...")
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            if algoName == targetAlgorithm:
                file = open(location, "w")
                for i in range(0,len(fig.data[traceIds[algoName+"-"+voice]].y)):
                    file.write(str(fig.data[traceIds[algoName+"-"+voice]].x[i])+" "+str(fig.data[traceIds[algoName+"-"+voice]].y[i]))
                    file.write("\n")
                file.close()
                print("Saved", location)
            
    def on_button_undo(x):
        global undoState
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            fullName = algoName+"-"+voice
            if algoName == "crepe":                
                newData = np.array(undoState[traceIds[algoName+"-"+voice]].y)          
                fig.data[traceIds[algoName+"-"+voice]].y = newData
    
    def on_button_deselect(x):
        global undoState
        for scatter in fig.data:                   
            fig.data[traceIds[algoName+"-"+voice]].selectedpoints = None
            fig.data[traceIds[algoName+"-"+voice]].selectedpoints = []
    
    def on_button_range(x):
        global selectionXrange, selectionYrange
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            location = location.replace("pitches-vuv-new", "pitch-corrections")
            selectionXrange[0] = round(float(selectionXrange[0]),2)
            selectionXrange[1] = round(float(selectionXrange[1]),2)
            selectionYrange[0] = round(float(selectionYrange[0]),2)
            selectionYrange[1] = round(float(selectionYrange[1]),2)
            if algoName == "crepe" and scatter.visible == True:
                
                with open(location,"r") as f:
                    currentLines = []
                    num_lines = 0
                    allLines = f.readlines()
                    if len(allLines) == 0:
                        currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                    else:
                        for i in range(0, len(allLines)):
                            num_lines = i
                            line = allLines[i]
                            line = line.replace("\n","")
                            lineArray = line.split(" ")
                            lineArray = [float(numeric_string) for numeric_string in lineArray]

                            if lineArray[0] < selectionXrange[0] < lineArray[1] or lineArray[0] < selectionXrange[1] < lineArray[1]:
                                #print("# overlapping")
                                if lineArray[0] < selectionXrange[0] < lineArray[1] and lineArray[0] < selectionXrange[1] < lineArray[1]: 
                                    # in between
                                    currentLines.append(str(lineArray[0])+" "+str(selectionXrange[0])+" "+str(lineArray[2])+" "+str(lineArray[3]))
                                    currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                                    currentLines.append(str(selectionXrange[1])+" "+str(lineArray[1])+" "+str(lineArray[2])+" "+str(lineArray[3]))
                                    break
                                elif lineArray[0] < selectionXrange[0] < lineArray[1]:
                                    currentLines.append(str(lineArray[0])+" "+str(selectionXrange[0])+" "+str(lineArray[2])+" "+str(lineArray[3]))
                                    currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                                    break
                                    # interval is on the right
                                elif lineArray[0] < selectionXrange[1] < lineArray[1]:
                                    currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                                    currentLines.append(str(selectionXrange[1])+" "+str(lineArray[1])+" "+str(lineArray[2])+" "+str(lineArray[3]))
                                    break
                                    # interval is on the left
                            elif selectionXrange[0] < lineArray[0] and selectionXrange[1] > lineArray[1]:
                                #print("# covering")
                                if i+1 < len(allLines):
                                    nextLine = allLines[i+1]
                                    nextLine = nextLine.replace("\n","")
                                    nextLineArray = nextLine.split(" ")
                                    nextLineArray = [float(numeric_string) for numeric_string in nextLineArray]
                                    
                                    if nextLineArray[0] > selectionXrange[1]:
                                        currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                                        break
                            elif lineArray[1] < selectionXrange[0]:
                                #print("# not overlapping -- greater")
                                if i+1 < len(allLines):
                                    nextLine = allLines[i+1]
                                    nextLine = nextLine.replace("\n","")
                                    nextLineArray = nextLine.split(" ")
                                    nextLineArray = [float(numeric_string) for numeric_string in nextLineArray]

                                    if nextLineArray[0] > selectionXrange[1]:
                                        # not overlapping
                                        currentLines.append(line)
                                        currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                                        break
                                    else:
                                        currentLines.append(line)
                                        #currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                                        #currentLines.append(str(selectionXrange[1])+" "+str(nextLineArray[1])+" "+str(nextLineArray[2])+" "+str(nextLineArray[3]))
                                        #i += 1
                                else:
                                    currentLines.append(line)
                                    currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                                    break

                            elif lineArray[0] > selectionXrange[1]:
                                #print("# not overlapping -- smaller")
                                if i != 0:
                                    previousLine = allLines[i-1]
                                    previousLine = previousLine.replace("\n","")
                                    previousLineArray = previousLine.split(" ")
                                    previousLineArray = [float(numeric_string) for numeric_string in previousLineArray]
                                    
                                    if previousLineArray[1] < selectionXrange[0]:
                                        currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                                        currentLines.append(line)
                                        break
                                    else:
                                        currentLines.append(line)
                                else:
                                    currentLines.append(str(selectionXrange[0])+" "+str(selectionXrange[1])+" "+str(selectionYrange[0])+" "+str(selectionYrange[1]))
                                    currentLines.append(line)
                                    break
                                    
                        for i in range (num_lines+1,len(allLines)):
                            # add rest of them.
                            line = allLines[i]
                            line = line.replace("\n","")
                            currentLines.append(line)

                    
                           
                with open(location,"w") as f:
                    print(currentLines)
                    for line in currentLines:
                        if(line != ""):
                            #print(line)
                            f.write(line)
                            f.write("\n")
                    
                print("Saved Range", location)
            
    button_postprocess = Button(description=f"Post-process {targetAlgorithm}", button_style='danger')
    button_octave = Button(description="Correct note", button_style='danger')
    button_octave_selected = Button(description="Correct selected", button_style='danger')
    button_delete = Button(description="Delete selected", button_style='danger')
    button_delete_note = Button(description="Delete note", button_style='danger')
    button_save = Button(description=f"Save {targetAlgorithm}", button_style='danger')
    button_undo = Button(description="Undo", button_style='danger')
    button_deselect = Button(description="Deselect", button_style='danger')
    button_range = Button(description="Set Range", button_style='danger')
    button_postprocess.on_click(on_button_postprocess)
    button_octave.on_click(on_button_octave)
    button_octave_selected.on_click(on_button_octave_selected)
    button_delete.on_click(on_button_delete)
    button_delete_note.on_click(on_button_delete_note)
    button_save.on_click(on_button_save)
    button_undo.on_click(on_button_undo)
    button_deselect.on_click(on_button_deselect)
    button_range.on_click(on_button_range)

    #button_next.observe(on_button_next, )

    display(widgets.VBox([fig,HBox([button_postprocess, button_delete, button_delete_note, button_octave_selected, button_octave]), HBox([button_save, button_undo, button_deselect, button_range])]))
    

### Can Correct

- Crepe (target algo) notes that should be 0 (usually another voice is singing)
- Crepe (target algo) notes for which another algorithm has a better estimate (in another octave)

### Cannot Correct

- consonants that show up as 0's or other notes (usually no algorithm has a good estimate)
- notes in the wrong octave but for which no algorithm has a good estimate in the right octave
- notes in the right octave that sound wrong 



In [ ]:
graph(working_song,'crepe', False)

#display(VBox([button_delete,button_next]))

## Problem sections

### Problems:

- partly-corrected notes
- 0's (probably from Voiced-Unvoiced algorithm) that should be notes

### Non-problems:

- consonants that show up as 0's or other notes
- notes that sound wrong but for which no algorithm offers a correction

#### Syntax

- start_time, algorithm, problem
- first_start_time-last_start_time, algorithm, problem (if there are 2 or more sections in succession)

### Gurian songs

#### Mok'le Mravalzhamieri

Bass

Middle - crosses top a lot

Top

- 3.2, 4.0, 9.5, 10.1, 11.6, 17.4, 19.7 crepe estimates 4th or 5th too high (Noll mostly correct) 
- 18.7 crepe estimate 5th too high, miscorrected (Noll)

#### Mts'vanesa Da Ukudosa

Bass

- 43.9-45.9, crepe 0
- 87.8-88.4, crepe 0
- 129.4-129.8, crepe 0

Middle - corrected consonants when top voice present

Top - corrected consonants when bottom voice present

- 15.6, crepe bass note
- 29.0, 32.2, 39.5, 72.4, 82.4, 102.8, 110.8, 113.5, 123.1, 126.3 crepe deleted mystery estimates during apparent silence
- 70.4, crepe bass note
- 83.5, crepe bass note (top note one octave up, all algorithms pick the bass note)
- 102.1, crepe bass note

#### Nanina (1)

(Lost notes to git?)

#### Nanina (2)

Bass

Middle

Top

- 56.0, crepe estimate is another voice
- 66.7, crepe miscorrected short note to 0

#### Orira

Bass

Middle - hard to tell when the estimate is a bass note throughout; Noll better in some passages

Top - some estimates are other voices: 21.3, 46.0, 63.6, 77.7 (no better estimate)

- 87.3, crepe corrected other voice to a note above Boersma; didn't correct 87.8, 88.4

#### P'at'ara Saq'varelo

Bass

Middle - hard to tell when the estimate is a bass note throughout

- 68.4, crepe consonant changed to 0 because estimate is bass note

Top 

- 

#### Pikris Simghera

Bass

Middle

Top

#### Sabodisho

Bass

- 106.7-107.3, crepe 1 octave low

Middle

- 86.3, crepe 1 octave low
- 92.9-93.7 crepe 0
- 105.3, crepe 0
- 116.1-116.6, crepe 0
- 128.7-130.2, crepe 0
- 148.5-149.8, crepe 0
- 168.3-169.3, crepe 0
- 194.2-197.6, crepe 0

Top

### Megrelian Songs

In [ ]:
import glob
for file in glob.iglob("/Users/kutlay/Documents/GitHub/Voice/data/pitches-crepe-range" + '/**/*.txt', recursive=True):
    file = open(file, "w")
    file.write("")
    file.close()